# LIME Examples

In [ ]:
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
import transformers
import matplotlib.pyplot as plt
import seaborn as sns
from apex import amp
from lime.lime_text import LimeTextExplainer
from pprint import pprint
import dill

In [ ]:
# set path to media-bias-prediction repository 
repo_path = os.path.dirname(os.getcwd())
os.chdir(os.path.join(repo_path, 'other_notebooks'))

## Loading and preparing data

In [ ]:
os.chdir(os.path.join(repo_path, 'data_preparation','allsides_data'))

bias_test = torch.load('allsides_bias_test.pt')
text_test = torch.load('allsides_contents_text_test.pt')
mask_test = torch.load('allsides_contents_mask_test.pt')

os.chdir(os.path.join(repo_path, 'other_notebooks'))

In [ ]:
os.chdir(os.path.join(repo_path, 'data_preparation','allsides_data'))

allsides_source_test = np.load('allsides_source_test.npy', allow_pickle=True).flatten()

# sources to be removed:
wrongly_labeled = ['RightWingWatch']
news_aggregators = ['Drudge Report', 'Real Clear Politics', 'Yahoo News', 'RightWingWatch'] 
tabloids = ['New York Daily News', 'Daily Mail', 'New York Post'] 
unwanted_sources = wrongly_labeled + news_aggregators + tabloids

# creating boolean array to mark unwanted sources
boolean_array_test = np.full((len(allsides_source_test), ), False)

for source in unwanted_sources:
    boolean_array_test += allsides_source_test==source 

# boolean to remove aggregators
inverted_boolean_array_test = np.invert(boolean_array_test)

# bias
bias_test = bias_test[inverted_boolean_array_test]

# text and masks
text_test = text_test[inverted_boolean_array_test]
mask_test = mask_test[inverted_boolean_array_test]

# sources
allsides_source_test = allsides_source_test[inverted_boolean_array_test]

os.chdir(os.path.join(repo_path, 'other_notebooks'))

In [ ]:
test_set = TensorDataset(text_test, mask_test, bias_test)

## Model class

In [ ]:
class Model(nn.Module):
    def __init__(self, hidden_size, num_labels, droput_prob, bert_model_module, output_attentions=False, pooled_output = True):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_labels = num_labels
        self.output_attentions = output_attentions
        self.pooled_output = pooled_output

        self.bert = bert_model_module
        self.dropout = nn.Dropout(dropout_prob)
        self.linear = nn.Linear(hidden_size,hidden_size)
        self.tanh = nn.Tanh()

        self.classifier_layer = nn.Linear(hidden_size, num_labels) 
        
    def forward(self, text, mask):
        # token_type_ids and position_ids are created automaticly 
        bert_out = self.bert(input_ids = text, attention_mask = mask)
        
        if self.pooled_output:
            # Choosing only CLS token output and apply linear layer + TanH 
            pooled_out = bert_out[1]
            # Applying dropout
            pooled_out = self.dropout(pooled_out)

            out = self.classifier_layer(pooled_out)
        else:
            ### Last Layer average
            # summing up over sequence lenght and devide by unmasked sequence length 
            # resulting in tensor with shape (batch_size,hidden_size)
            last_layer = torch.sum(bert_out[0], dim=1)/torch.sum(mask,dim=1).reshape([len(mask),1])
            last_layer = self.tanh(self.linear(last_layer))
            last_layer = self.dropout(last_layer)
            out = self.classifier_layer(last_layer)
               
        # Saving attention layer outputs if set True
        if self.output_attentions:
            out = out, bert_out[2]
        
        return out

## Preparing model

### Parameters

In [ ]:
##### Loading Bert 
BertModel = transformers.BertModel

### Device to run model on, either GPU or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Model inputs
hidden_size = 768
num_labels = 5 
dropout_prob = 0.1
cost_sensitive = False

### Hyperparameters
batch_size = 16 
learning_rate = 2e-5
### Use of nvidia apex for mixed precession calculations
mixed_precision = True 

In [ ]:
##### Initilize and configure Bert
bert_model = BertModel.from_pretrained('bert-base-uncased') 

##### Initilize model 
model = Model(hidden_size, num_labels, dropout_prob, bert_model, pooled_output=True).to(device)

### Optimizer, choosing learning rate 
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
### Applying mixed precision 
if mixed_precision:
    model, optimizer = amp.initialize(model, optimizer, opt_level="O1") 
### Loss function
if cost_sensitive:
    loss_fct = CostSensitiveCELoss(num_labels).to(device)
else: 
    loss_fct = nn.CrossEntropyLoss().to(device)

### Loading weights

In [ ]:
checkpoint = torch.load(os.path.join(repo_path,'deep_learning_models', 'weights', 'amp_checkpoint_allsides_aggregators_tabloids_duplicates_removed_rerun_1_epoch3.pt'))

model.load_state_dict(checkpoint['model'])

## Applying LIME

### Setting up variables

In [ ]:
# Loading and initilizing tokenizer
BertTokenizer = transformers.BertTokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
lime_text = text_test
true_labels = ['Left', 'Lean Left', 'Center', 'Lean Right', 'Right']
true_values = bias_test.numpy()
lime_predict_fct__mask = mask_test
num_plots = len(true_labels)
predicted_labels = true_labels
palette = {'Left': 'blue', 'Lean Left': 'slateblue', 'Center': 'grey', 'Lean Right': 'indianred', 'Right': 'red'}


In [ ]:
def prediction_fct(text_input):
    '''
    Takes in list of strings, where each string is a whole article
    
    Returns numpy array containing predicted probabilites of shape (d,k)
    where d is the number of article samples given and k is the number of classes (5)
    '''
    prob_list = []
    for sample in text_input:
        token_list = sample.split(' ')[:-1]
        text = torch.tensor(bert_tokenizer.convert_tokens_to_ids(token_list)).unsqueeze(0)
        
    
        with torch.no_grad():
            model.eval()
            mask = lime_predict_fct__mask[index].clone().detach().unsqueeze(0)  
            text, mask = text.to(device), mask.to(device)
            output = model(text, mask)
            sample_probabilities = F.softmax(output,1).cpu().numpy()
            prob_list.append(sample_probabilities)    

    probabilities = np.concatenate(prob_list)
    return probabilities

### Selecting article

In [ ]:
### choose article index# 
index = 997 #340 #15    #
#########################
# copy tensor
text_tensor = lime_text[index].clone().detach() 
# create string to feed into LIME explain_instance
text_tokens = bert_tokenizer.convert_ids_to_tokens(text_tensor)
text_string = ''.join([token + ' ' for token in text_tokens])
# look at article 
print(f'True Label: {true_labels[int(true_values[index])]}   Source: {allsides_source_test[index]}')
print('-----------------------------------------------------------------------------') 
pprint(text_string)


### Running LIME

In [ ]:
# create explaination object
explainer = LimeTextExplainer(class_names=true_labels)
exp = explainer.explain_instance(text_string, prediction_fct, num_features=10, labels=[0,1,2,3,4], num_samples=10000) 

predected_probs = prediction_fct([text_string])
print(f'Predicted class: {predicted_labels[np.argmax(predected_probs)]} ,')
print(f'True class: {true_labels[int(true_values[index])]}') 
print(f'Probabilites in %: {predected_probs[0,0]*100:.4} {predicted_labels[0]}, {predected_probs[0,1]*100:.4} {predicted_labels[1]}, '
      f'{predected_probs[0,2]*100:.4} {predicted_labels[2]}, {predected_probs[0,3]*100:.4} {predicted_labels[3]}, {predected_probs[0,4]*100:.4} {predicted_labels[4]} ') 

# extracting lists with word,probability tuples from lime explaination object
word_prob_pairs_left = exp.as_list(label=0)
word_prob_pairs_moderate_left = exp.as_list(label=1)
word_prob_pairs_center = exp.as_list(label=2) 
word_prob_pairs_moderate_right = exp.as_list(label=3)
word_prob_pairs_right = exp.as_list(label=4) 

### Presenting outcome

In [ ]:
# creating dataframe for easier readability
lime_left_df = pd.DataFrame(word_prob_pairs_left, columns=['words left', 'probabilities left (%)'])
lime_left_df['probabilities left (%)'] = np.round(lime_left_df['probabilities left (%)']*100,4)

lime_moderate_left_df = pd.DataFrame(word_prob_pairs_moderate_left, columns=['words lean left', 'probabilities lean left (%)'])
lime_moderate_left_df['probabilities lean left (%)'] = np.round(lime_moderate_left_df['probabilities lean left (%)']*100,4)

lime_center_df = pd.DataFrame(word_prob_pairs_center, columns=['words center', 'probabilities center (%)'])
lime_center_df['probabilities center (%)'] = np.round(lime_center_df['probabilities center (%)']*100,4)

lime_moderate_right_df = pd.DataFrame(word_prob_pairs_moderate_right, columns=['words lean right', 'probabilities lean right (%)'])
lime_moderate_right_df['probabilities lean right (%)'] = np.round(lime_moderate_right_df['probabilities lean right (%)']*100,4)

lime_right_df = pd.DataFrame(word_prob_pairs_right, columns=['words right', 'probabilities right (%)'])
lime_right_df['probabilities right (%)'] = np.round(lime_right_df['probabilities right (%)']*100,4)

lime_df = pd.concat([lime_left_df, lime_moderate_left_df, lime_center_df, lime_moderate_right_df, lime_right_df], axis=1) # [lime_left_df, lime_center_df, lime_right_df] 
#lime_df

In [ ]:
### Plot LIME results
general_font_size = 'medium'
plt.rcParams.update({'font.size': 17})
lime_df_columns = list(lime_df.columns)

fig, axes = plt.subplots(1,num_plots, figsize=[24,5])

lime_max = 25  # 15
lime_min = -25 #-15

for i in range(num_plots):
    sns.barplot(list(lime_df.iloc[:,i*2+1]), list(lime_df.iloc[:,i*2]), color=palette[predicted_labels[i]], ax=axes[i])
    axes[i].set_xlim([lime_min,lime_max])
    axes[i].set_xticks([-20,-10,0,10,20]) # [-15,-10,-5,0,5,10,15] 
    axes[i].grid(True, axis='x')
    axes[i].set_title(true_labels[i])
    axes[i].set_xlabel('weights (in %)')

axes[0].set_ylabel('most relevant words')

fig.tight_layout()

In [ ]:
lime_example_name = 'abc_news_white_supremacist'

In [ ]:
fig.savefig(f'lime_{lime_example_name}_plot.png')


### Creating HTML file

In [ ]:
html_lime = exp.as_html(predict_proba=False, text=True, opacity=False)

with open(f"lime_{lime_example_name}.html", "w") as file:
    file.write(html_lime)

### Save LIME explanation object

In [ ]:
# save 
with open(f'lime_html_files/lime_{lime_example_name}_explanation', 'wb') as f:
    dill.dump(exp, f)

In [ ]:
# load
with open(f'lime_html_files/lime_{lime_example_name}explanation', 'rb') as f:
    exp = dill.load(f)